In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [13]:
class Action:
    BANK_LEFT = 'L'
    BANK_RIGHT = 'R'
    THRUST = 'T'
    FIRE = 'F'
    
    ACTIONS = [BANK_LEFT, BANK_RIGHT, THRUST, FIRE]

class Game:
    """A single game."""
    def __init__(self, actions, states, result):
        self.actions = actions
        self.states = states
        self.result = result
        
    def to_df(self):
        """Outputs a Pandas DF with one row for each time"""
        rows = []
        for act, state in zip(self.actions, self.states):
            rows.append({
                't': state['t'],
                't_int': round(state['t'] * 16),
                'x1': state['players']['0']['position'][0],
                'y1': state['players']['0']['position'][1],
                'vx1': state['players']['0']['velocity'][0],
                'vy1': state['players']['0']['velocity'][1],
                'a1': state['players']['0']['orientation'],
                'x2': state['players']['1']['position'][0],
                'y2': state['players']['1']['position'][1],
                'vx2': state['players']['1']['velocity'][0],
                'vy2': state['players']['1']['velocity'][1],
                'a2': state['players']['1']['orientation'],
                'bullets': [state['bullets'][k] for k in '01']
            })
        rows = pd.DataFrame(rows).set_index('t_int')
        rows['a1_0'] = rows['a1'] + wedge_size / 2
        rows['a1_1'] = rows['a1'] - wedge_size / 2
        rows['a2_0'] = rows['a2'] + wedge_size / 2
        rows['a2_1'] = rows['a2'] - wedge_size / 2
        return rows

In [19]:
width = 50 * 20
height = 50 * 20
player_r = 15
wedge_size = np.pi / 4
c = ['#4287f5', '#ffa30f']

In [20]:
def parse_player_actions(actions):
    return {act: act in actions for act in Action.ACTIONS}

def read_snoopy(filename):
    with open(filename, 'r') as stream:
        actiondata, statedata, resultdata = stream.read().split('\n\n')
    actions = []
    for line in actiondata.split('\n'):
        actions.append([parse_player_actions(acts) for acts in line.split(' ')])

    states = [json.loads(state) for state in statedata.split('\n')]
    res = int(resultdata.strip())
    return Game(actions, states, res)
    

In [21]:
g = read_snoopy("/home/nicholas/IdeaProjects/snoopy-server/replays/game1.snoopy")
df = g.to_df()
df.head()

,t,x1,y1,vx1,vy1,a1,x2,y2,vx2,vy2,a2,bullets,a1_0,a1_1,a2_0,a2_1
t_int,,,,,,,,,,,,,,,,
1,0.0625,658.875,666.125,-482.375,-61.500,3.375,341.125,332.875,482.375,0.875,0.25,"[[], []]",3.767699,2.982301,0.642699,-0.142699
2,0.1250,630.500,662.125,-378.250,-77.750,3.625,369.500,332.375,379.250,-28.875,0.50,"[[], []]",4.017699,3.232301,0.892699,0.107301
3,0.1875,608.000,657.000,-311.625,-93.875,3.875,398.875,334.000,685.375,197.625,0.75,"[[], []]",4.267699,3.482301,1.142699,0.357301
4,0.2500,589.375,650.875,-264.500,-109.750,4.125,443.625,350.000,755.125,429.125,1.00,"[[], []]",4.517699,3.732301,1.392699,0.607301
5,0.3125,573.375,643.750,-228.875,-125.125,4.375,489.875,380.000,670.750,621.375,1.25,"[[], []]",4.767699,3.982301,1.642699,0.857301


In [22]:
names = ['Player 1', 'Player 2']
def parse_bullets(bullets):
    rows = []
    for p_bullets, name in zip(bullets, names):
        for bullet in p_bullets:
            rows.append({
                'x': bullet['position'][0],
                'y': bullet['position'][1],
                'player': name
            })
    return pd.DataFrame(rows)

In [23]:
parse_bullets(df.iloc[-1]['bullets'])

,x,y,player
0,618.750,653.250,Player 1
1,669.750,632.875,Player 1
2,709.000,594.500,Player 1
3,730.500,544.125,Player 1
4,731.250,489.250,Player 1
5,596.000,502.750,Player 2
6,545.000,523.125,Player 2
7,505.750,561.500,Player 2
8,484.125,612.000,Player 2
9,483.500,666.875,Player 2


In [24]:
from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Slider, TextInput, Button
from bokeh.plotting import figure
from bokeh.plotting import show
from bokeh.io import output_notebook
from bokeh.models import CategoricalColorMapper



output_notebook()

def app(doc):
    
    player_map = CategoricalColorMapper(factors=names, palette=c)

    t = 1
    scale = 1 / 1.5
    source = df.loc[[t], :]
    
    
    def plot(t):
        # Set up plot
        plot = figure(height=round(height * scale), width=round(width * scale), title="Game",
                      tools="crosshair,save",
                      x_range=[0, width], y_range=[0, height])

        players = df.loc[[t], :]
        bullets = parse_bullets(df.loc[t, 'bullets'])
        plot.wedge('x1', 'y1', start_angle='a1_0', end_angle='a1_1', color=c[0], radius=player_r, legend_label='Player 1', source=players)
        plot.wedge('x2', 'y2', start_angle='a2_0', end_angle='a2_1', color=c[1], radius=player_r, legend_label='Player 2', source=players)
        if not bullets.empty:
            plot.circle('x', 'y', color={'field': 'player', 'transform': player_map}, source=bullets)
        return plot


    # Set up widgets
    time = Slider(value=min(df.index), start=min(df.index), end=max(df.index), step=1)

    def update_data(attrname, old, new):
        col.children[0] = plot(time.value)

    time.on_change('value', update_data)
    
    playing = False
    def animate_update():
        global playing
        if 'playing' not in globals():
            pass
        elif playing:
            t = time.value + 1
            if t not in df.index:
                t = 1
            time.value = t

    callback_id = None

    def animate():
        global playing
        if button.label == '► Play':
            playing = True
            button.label = '❚❚ Pause'
        else:
            button.label = '► Play'
            playing = False
            
    callback_id = doc.add_periodic_callback(animate_update, 200)

    button = Button(label='► Play', width=40)
    button.on_click(animate)


    # Set up layouts and add to document
    col = column(plot(1), row(time, button))
    

    doc.add_root(col)
    doc.title = "Game"
    
show(app)

Loading BokehJS ...